In [1]:
import os
os.environ["PYTHON_EXEC"] = "/Users/jessieyu/.pyenv/versions/qka-demo/bin/python3"
os.environ["NTC_DOC_FILE"] = "runtime/qka_doc.json"
os.environ["NTC_PROGRAM_FILE"] = "runtime/qka_runtime_outer.py"

In [2]:
from qiskit import IBMQ

In [3]:
provider = IBMQ.load_account()

In [4]:
provider.runtime.programs()

Runtime Program Quantum-Kernel-Alignment:
  Description: Quantum kernel alignment algorithm that learns, on a given dataset, a quantum kernel maximizing the SVM classification margin.
  Parameters:
    feature_map:
      description: An instance of FeatureMapForrelation in dictionary format used to map classical data into a quantum state space.
      type: dict
    data:
      description: NxD array of training data, where N is the number of samples and D is the feature dimension.
      type: numpy.ndarray
    labels:
      description: Nx1 array of +/-1 labels of the N training samples.
      type: numpy.ndarray
    lambda_initial:
      description: Initial parameters of the quantum feature map.
      type: numpy.ndarray
    spsa_steps:
      description: Number of SPSA optimization steps.
      type: int
    C:
      description: Penalty parameter for the soft-margin support vector machine.
      type: float
  Returns:
    best_kernel_parameters:
      description: The optimized ker

In [5]:
import numpy as np
from qka.featuremaps import FeatureMapForrelation

num_samples=5  # number of samples per class in the input data
num_features=2 # number of features in the input data
depth=2        # depth of the feature map circuit
C=1            # SVM soft-margin penalty
spsa_steps=2   # number of SPSA iterations

# Define the feature map and its initial parameters:
fm = FeatureMapForrelation(feature_dimension=num_features, depth=depth, entangler_map=None)
lambda_initial = np.random.uniform(-1,1, size=(fm._num_parameters))

# create random test data and labels:
x_train = np.random.rand(2*num_samples, num_features)
y_train = np.concatenate((-1*np.ones(num_samples), np.ones(num_samples)))

backend = provider.backend.ibmq_qasm_simulator

In [6]:
def interim_result_callback(interim_result):
    print(f"interim result: {interim_result}\n")

In [7]:
# Call QKA runtime program.
runtime_params = {
    'feature_map': fm.to_dict(),
    'data': x_train,
    'labels': y_train,
    'lambda_initial': lambda_initial,
    'spsa_steps': 2,
    'C': C
}
result = provider.runtime.run(program_name="Quantum-Kernel-Alignment",
                              backend=backend,
                              params=runtime_params,
                              callback=interim_result_callback,
                             ).result()
print("final result:")
print(f"aligned_kernel_parameters: {result['aligned_kernel_parameters']}")
print(f"aligned_kernel_matrix: {result['aligned_kernel_matrix']}")

interim result: {'cost': 7.489755896607097, 'lambda': array([-0.9538571 , -0.80595599, -0.56554543,  0.03199335]), 'cost_plus': 7.2111320834159365, 'cost_minus': 7.768379709798256, 'cost_final': 7.489755896607097}

interim result: {'cost': 7.391245631532837, 'lambda': array([-0.95825878, -0.81035767, -0.56114375,  0.03639503]), 'cost_plus': 7.453537990927787, 'cost_minus': 7.328953272137888, 'cost_final': 7.391245631532837}

final result:
aligned_kernel_parameters: [-0.19121159 -0.16163137 -0.11266892  0.00683884]
aligned_kernel_matrix: [[1.         0.359375   0.11621094 0.52929688 0.07226562 0.07519531
  0.31835938 0.01074219 0.15234375 0.37011719]
 [0.359375   1.         0.1328125  0.86132812 0.73632812 0.25390625
  0.125      0.125      0.10058594 0.95214844]
 [0.11621094 0.1328125  1.         0.0390625  0.41601562 0.2890625
  0.80859375 0.40917969 0.99414062 0.06445312]
 [0.52929688 0.86132812 0.0390625  1.         0.47167969 0.03808594
  0.09960938 0.00292969 0.0234375  0.8125    